In [ ]:
##### imports ##########

In [1]:
import numpy as np
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
import pandas as pd

In [3]:
# C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/

In [19]:
! ls -lrt C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/*.parquet | wc -l

12


In [21]:
! ls C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/*.parquet

C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/april.parquet
C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/august.parquet
C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/december.parquet
C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/february.parquet
C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/january.parquet
C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/july.parquet
C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/june.parquet
C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/march.parquet
C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/may.parquet
C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/november.parquet
C:/Users/SRA

In [ ]:
##### Create Cluster ##########

In [25]:
# LocalCluster?

In [22]:
cluster = LocalCluster(
    name='timeseries',
    n_workers=2,
    threads_per_worker=1,
    host='localhost',
    scheduler_port=8786,
    asynchronous=True, 
    processes=False
)
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 7.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50572,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 7.88 GiB
Comm: tcp://127.0.0.1:50593,Total threads: 1
Dashboard: http://127.0.0.1:50599/status,Memory: 1.97 GiB
Nanny: tcp://127.0.0.1:50575,


In [ ]:
############## Steps ####################

In [26]:
def optimeToFitMemory(ddf):
    new_types = dict(
        int32 = ['passenger_count',],
        int16 = ['VendorID',]
    )
    for key in new_types:
        for col in new_types[key]:
            ddf[col] = ddf[col].astype(key)
    return ddf

In [28]:
ddf = dd.read_parquet('C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/january.parquet', npartitions=4)
s1 = ddf.loc[
    ddf.tpep_pickup_datetime>'2021-12-31 23:00:00', 
    ['tpep_pickup_datetime','passenger_count', 'VendorID']
]
s2 = s1.set_index('tpep_pickup_datetime', sorted=True)
s3 = s2.ffill()
s4 = s3.resample('H').agg({'passenger_count': 'sum', 'VendorID': 'count'})
s5 = s4.map_partitions(optimeToFitMemory)
s6 = s5.reset_index()
s6.to_parquet('C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022.parquet', append=True, ignore_divisions=True)

In [29]:
s6.dask

HighLevelGraph with 12 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x198527c07c0>
 0. read-parquet-2462bd2dd4201e09902d669a7bceda40
 1. getitem-19c27aac1e13aa650ad9b92320aa89d9
 2. gt-9edb7a87808d7cfd20b282c5f075cd8d
 3. loc-series-de49021bf8b80b1ac59d0d144d236c5f
 4. set_index-51b0cd53e74286cedc200101c83cb4bb
 5. fillna-chunk-73937cb4e9d7592f6a63b6f7b663b53c
 6. overlap-concat-c34c27c894a12d5326d6603d6f717f51
 7. overlap-ffill-31f5f403a41eb11d546e0b851332d2d1
 8. repartition-merge-3d7b7dab9b46dae5c19780d6fef8642d
 9. resample-b3aeea2db0485977f92084991e2dd0fa
 10. optimeToFitMemory-de0ad49c0351876232263a3569a9cdff
 11. reset_index-9f423ab1559fd44394af569561271fb6

In [31]:
files = [
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/april.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/august.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/december.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/february.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/january.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/july.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/june.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/march.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/may.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/november.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/october.parquet',
    'C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022/september.parquet',
]

In [32]:
optimized_df = pd.DataFrame(
    {
        'tpep_pickup_datetime': [pd.Timestamp(year=2022, month=1, day=1, hour=0, minute=0, second=0)],
        'passenger_count':[0,],
        'VendorID': [0,]
    }
)


for file in files:
    ddf = dd.read_parquet(file, npartitions=100)
    s1 = ddf.loc[
        ddf.tpep_pickup_datetime >= '2022-01-01 00:00:00', 
        ['tpep_pickup_datetime','passenger_count', 'VendorID']
    ]
    s2 = s1.set_index('tpep_pickup_datetime', sorted=True)
    s3 = s2.ffill()
    s4 = s3.resample('H').agg({'passenger_count': 'sum', 'VendorID': 'count'})
    s5 = s4.map_partitions(optimeToFitMemory)
    s6 = s5.reset_index()
    optimized_df = pd.concat([optimized_df, s6.compute()], axis=0, ignore_index=True)

In [34]:
optimized_df.shape

(21626, 3)

In [35]:
optimized_df.head()

,tpep_pickup_datetime,passenger_count,VendorID
0,2022-01-01 00:00:00,0,0
1,2022-03-31 15:00:00,4,4
2,2022-03-31 16:00:00,3,3
3,2022-03-31 17:00:00,0,0
4,2022-03-31 18:00:00,0,0


In [36]:
optimized_df.tail()

,tpep_pickup_datetime,passenger_count,VendorID
21621,2022-09-30 20:00:00,9959,6905
21622,2022-09-30 21:00:00,10046,6894
21623,2022-09-30 22:00:00,10244,6999
21624,2022-09-30 23:00:00,9633,6547
21625,2022-10-01 00:00:00,47,19


In [37]:
optimized_df.isnull().sum()

tpep_pickup_datetime    0
passenger_count         0
VendorID                0
dtype: int64

In [38]:
optimized_df.drop([0, 21625], inplace=True)

In [39]:
optimized_df.to_csv('C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022.csv', index=False)

In [40]:
pd.read_csv('C:/Users/SRA/Desktop/backup/C/MLgrit/time_series_project/uber-taxi-demand/data/2022.csv', encoding='unicode_escape')

,tpep_pickup_datetime,passenger_count,VendorID
0,2022-03-31 15:00:00,4,4
1,2022-03-31 16:00:00,3,3
2,2022-03-31 17:00:00,0,0
3,2022-03-31 18:00:00,0,0
4,2022-03-31 19:00:00,0,0
...,...,...,...
21619,2022-09-30 19:00:00,12107,8576
21620,2022-09-30 20:00:00,9959,6905
21621,2022-09-30 21:00:00,10046,6894
21622,2022-09-30 22:00:00,10244,6999
